# 운영 로직
## 기사가 사진을 업로드하면 자동실행 코드

In [ ]:
pip install watchdog

In [ ]:
# ============================================================
# ✅ uploads/{index} 감시 → 심각도 자동 실행
# ============================================================

import os, time, re, glob
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# 위에서 만든 함수 import
# from severity_infer import infer_severity_for_index

UPLOAD_ROOT = "uploads"

def is_ready(index_folder: str) -> bool:
    """before_* 와 after_* 쌍이 하나라도 있으면 준비 완료"""
    has_before = len(glob.glob(os.path.join(index_folder, "before_*.*"))) > 0
    has_after  = len(glob.glob(os.path.join(index_folder, "after_*.*"))) > 0
    return has_before and has_after


class UploadHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return

        # uploads/{index}/xxx.jpg → index 추출
        m = re.search(r"uploads[/\\]([^/\\]+)[/\\]", event.src_path)
        if not m:
            return
        index = m.group(1)
        folder = os.path.join(UPLOAD_ROOT, index)

        if not is_ready(folder):
            return

        try:
            print(f"[AUTO] infer start: {index}")
            infer_severity_for_index(index)
            print(f"[AUTO] infer done : {index}")
        except Exception as e:
            # 업로드 중일 수 있음 → 다음 이벤트에서 다시 실행됨
            print(f"[AUTO] skip {index}: {e}")


if __name__ == "__main__":
    os.makedirs(UPLOAD_ROOT, exist_ok=True)

    observer = Observer()
    observer.schedule(UploadHandler(), UPLOAD_ROOT, recursive=True)
    observer.start()

    print("🚗 waiting for 기사 uploads...")

    try:
        while True:
            time.sleep(1)
    finally:
        observer.stop()
        observer.join()
